## Advance Data Manipulation in Pandas


#### Group By And Pivot Aggregation 


##### Group By

In [25]:
#Way 1 only one aggregation function
import pandas as pd
df=pd.read_csv("data.csv")
grouped_df=df.groupby("Product")[["Value","Sales"]].mean()
display(grouped_df)

#Way 2 multiple aggregation functions
grouped_df1=df.groupby("Product")[["Value","Sales"]].agg(["mean","sum"])
print(grouped_df1.columns.values)#list of tuples
#changing column names from multi index to single index
grouped_df1.columns = ['_'.join(col).strip() for col in grouped_df1.columns.values]
display(grouped_df1.reset_index().head(5))

#Way 3 Different aggregation functions for different columns
grouped_df2=df.groupby("Product").agg({"Value":"mean","Sales":"sum"})
display(grouped_df2.reset_index())

##Way4 

grouped_df3=df.groupby("Product").agg({"Value":["mean","count"],"Sales":"sum"})
#To set index
#df.set_index('Product',inplace=True)
display(grouped_df3.reset_index(inplace=True))
display(grouped_df3[[("Value","mean"),("Value","count")]].head(5))

,Value,Sales
Product,,
Product1,46.214286,574.866667
Product2,52.800000,567.230769
Product3,55.166667,535.055556


[('Value', 'mean') ('Value', 'sum') ('Sales', 'mean') ('Sales', 'sum')]


,Product,Value_mean,Value_sum,Sales_mean,Sales_sum
0,Product1,46.214286,647.0,574.866667,8623.0
1,Product2,52.800000,792.0,567.230769,7374.0
2,Product3,55.166667,993.0,535.055556,9631.0


,Product,Value,Sales
0,Product1,46.214286,8623.0
1,Product2,52.800000,7374.0
2,Product3,55.166667,9631.0


None

Value      
        mean count
0  46.214286    14
1  52.800000    15
2  55.166667    18

In [ ]:
## Best Way 

display(df.head(5))
grouped_df1 = df.groupby(["Product","Region"]).agg(
    Value_Avg=("Value", "mean"),
    Value_Total=("Value", "sum"),
    Sales_Avg=("Sales", "mean"),
    Sales_Total=("Sales", "sum")
)
display(grouped_df1.head(5))
display(grouped_df1.reset_index().head(5)) 

,Date,Category,Value,Product,Sales,Region
0,2023-01-01,A,28.0,Product1,754.0,East
1,2023-01-02,B,39.0,Product3,110.0,North
2,2023-01-03,C,32.0,Product2,398.0,East
3,2023-01-04,B,8.0,Product1,522.0,East
4,2023-01-05,B,26.0,Product3,869.0,North


Value_Avg  Value_Total   Sales_Avg  Sales_Total
Product  Region                                                 
Product1 East    41.714286        292.0  600.714286       4205.0
         North    4.500000          9.0  868.500000       1737.0
         South   50.000000        100.0  673.000000       1346.0
         West    82.000000        246.0  333.750000       1335.0
Product2 East    28.000000         56.0  428.000000        856.0

,Product,Region,Value_Avg,Value_Total,Sales_Avg,Sales_Total
0,Product1,East,41.714286,292.0,600.714286,4205.0
1,Product1,North,4.500000,9.0,868.500000,1737.0
2,Product1,South,50.000000,100.0,673.000000,1346.0
3,Product1,West,82.000000,246.0,333.750000,1335.0
4,Product2,East,28.000000,56.0,428.000000,856.0


##### Pivot

To help solidify your understanding of how a Pivot design works, here is a summary of the two key architectural concepts we covered: the Physical Structure (how it looks) and the Data Access (how to get values out).1. The Design: The "L-Shape" MultiIndexWhen you pivot, you aren't just creating a table; you are creating a 2D Coordinate System. 

The design has two special labels called Axis Names that act as metadata.index.name (Vertical): Sits on the left. It labels the rows (e.g., Region).columns.name (Horizontal): Sits on top. 

It labels the columns (e.g., Product).Why it looks weird: These names aren't "data" in the rows; they are labels for the axes.

1 This is why they appear at different heights in your code output. You can create this look manually without a pivot by simply typing:df.index.name = 'Region' and df.columns.name = 'Product'.

2. Accessing the Design: Using .loc Even though your table has these "Axis Names" (Region, Product), you never use those names to look up data. You use the Labels inside them.

Think of it like a map: Region is the Y-axis and Product is the X-axis. To find a house, you need the specific coordinates.TaskCode SyntaxGet a single valuedf.loc['East', 'Apple'] Get a whole rowdf.loc['East', :] Get a whole columndf.loc[:, 'Apple']

In [8]:
import pandas as pd
df = pd.DataFrame({
    'Region': ['East', 'East', 'West', 'West', 'East'],
    'Product': ['A', 'B', 'A', 'B', 'A'],
    'Sales': [100, 150, 200, 50, 120]
})
display(df)
# Goal: Total Sales per Region for each Product
# aggfunc='sum' adds the two 'East/Product A' entries together
table = df.pivot_table(index='Region', columns='Product', values='Sales', aggfunc='sum')
#table.info()

display(table)
display(table.reset_index())
# Product    A    B
# Region           
# East     220  150
# West     200   50

,Region,Product,Sales
0,East,A,100
1,East,B,150
2,West,A,200
3,West,B,50
4,East,A,120


Product,A,B
Region,,
East,220,150
West,200,50


Product,Region,A,B
0,East,220,150
1,West,200,50


##### Pivot Advance

In [ ]:
import pandas as pd
import numpy as np

# 1. SETUP: Create sample data
data = {
    'Region': ['East', 'East', 'West', 'West', 'East', 'East', 'West', 'West'],
    'Manager': ['Alice', 'Bob', 'Charlie', 'Dave', 'Alice', 'Bob', 'Charlie', 'Dave'],
    'Year': [2024, 2024, 2024, 2024, 2025, 2025, 2025, 2025],
    'Product': ['Apple', 'Apple', 'Apple', 'Apple', 'Banana', 'Banana', 'Banana', 'Banana'],
    'Sales': [100, 150, 200, 250, 300, 350, 400, 450]
}
df = pd.DataFrame(data)

# 2. PIVOT: 2 Index levels, 2 Column levels
# This creates the "Nested" structure
pivot = df.pivot_table(
    index=['Region', 'Manager'], 
    columns=['Year', 'Product'], 
    values='Sales'
)
display(pivot)

# 3. RESET INDEX: Move 'Region' and 'Manager' from the index to columns
df_reset = pivot.reset_index()
print("--- Reset DataFrame ---")
display(df_reset)
# 4. ACCESS VALUE: Get the value for 2024 Apple 
# Note: Because columns are still a MultiIndex, we use a tuple
print("--- Extracting value ---")
apple_2024_sales = df_reset[(2024, 'Apple')] # columns act as tuples
display(apple_2024_sales)
# 5. JOIN COLUMNS: Flatten the MultiIndex headers into a single string
# We use a list comprehension to join Level 0 and Level 1 with an underscore
df_reset.columns = [
    f"{col[0]}_{col[1]}" if isinstance(col, tuple) and col[1] != "" 
    else col[0] 
    for col in df_reset.columns
]

# Display the final flattened result
print("--- Final Flattened DataFrame ---")
display(df_reset)

# Now you can access values using simple strings!
final_val = df_reset['2024_Apple']

Year             2024   2025
Product         Apple Banana
Region Manager              
East   Alice    100.0  300.0
       Bob      150.0  350.0
West   Charlie  200.0  400.0
       Dave     250.0  450.0

Year,Region,Manager,2024,2025
Product,,,Apple,Banana
0,East,Alice,100.0,300.0
1,East,Bob,150.0,350.0
2,West,Charlie,200.0,400.0
3,West,Dave,250.0,450.0


0    100.0
1    150.0
2    200.0
3    250.0
Name: (2024, Apple), dtype: float64

--- Final Flattened DataFrame ---


,Region,Manager,2024_Apple,2025_Banana
0,East,Alice,100.0,300.0
1,East,Bob,150.0,350.0
2,West,Charlie,200.0,400.0
3,West,Dave,250.0,450.0


#### Pivot VS Group By


In [ ]:
import pandas as pd
import numpy as np

# 1. CREATE RAW DATA
# Note: 'Value' has strings and a typo ('10o') to demonstrate cleaning
data = {
    'Date': ['2023-01', '2023-01', '2023-02', '2023-02', '2023-01'],
    'Region': ['North', 'South', 'North', 'South', 'North'],
    'Value': ['100', '150', '200', '50', '10o'], 
    'Units': [10, 15, 20, 5, 12]
}
df = pd.DataFrame(data)

# 2. CLEANING (The "Value" column and duplicates)
# Remove hidden spaces from column names
df.columns = df.columns.str.strip()

# Safely convert to numeric (coerce turns '10o' into NaN)
df["Value"] = pd.to_numeric(df["Value"], errors='coerce')

# Handle NaNs and duplicates
df["Value"] = df["Value"].fillna(df["Value"].mean())
df = df.drop_duplicates(keep='first')

# 3. TRANSFORMATION (Vectorized Math)
df["Revenue"] = df["Value"] * 1.1

# 4. GROUPBY (The "Long" Vertical Summary)
# Best for further calculations or programmatic use
gb_result = df.groupby(['Date', 'Region'])[['Revenue']].sum()

# 5. PIVOT TABLE (The "Wide" Grid Summary)
# Best for reports and human readability
pivot_result = df.pivot_table(
    index='Date', 
    columns='Region', 
    values='Revenue', 
    aggfunc='sum',
    margins=True,      # Adds 'All' (Grand Totals)
    fill_value=0       # Replaces empty combos with 0
)

# --- OUTPUTS ---
print("--- Cleaned DataFrame ---")
print(df)
print("\n--- GroupBy Output (Vertical List) ---")
display(gb_result)
#In Pandas, groupby returns a Series when you select only one column to calculate (like ['Revenue']) and perform a single math operation (like .sum()).
'''# Returns a Series
gb_series = df.groupby('Region')['Revenue'].sum()

# Returns a DataFrame
gb_df = df.groupby('Region')[['Revenue']].sum()'''

print((gb_result.reset_index()))

print("\n--- Pivot Table Output (Horizontal Grid) ---")
print(pivot_result)
print("================")
print(pivot_result.reset_index())

--- Cleaned DataFrame ---
      Date Region  Value  Units  Revenue
0  2023-01  North  100.0     10    110.0
1  2023-01  South  150.0     15    165.0
2  2023-02  North  200.0     20    220.0
3  2023-02  South   50.0      5     55.0
4  2023-01  North  125.0     12    137.5

--- GroupBy Output (Vertical List) ---


Revenue
Date    Region         
2023-01 North     247.5
        South     165.0
2023-02 North     220.0
        South      55.0

      Date Region  Revenue
0  2023-01  North    247.5
1  2023-01  South    165.0
2  2023-02  North    220.0
3  2023-02  South     55.0

--- GroupBy Output (after unstack) ---
        Revenue       
Region    North  South
Date                  
2023-01   247.5  165.0
2023-02   220.0   55.0

--- Pivot Table Output (Horizontal Grid) ---
Region   North  South    All
Date                        
2023-01  247.5  165.0  412.5
2023-02  220.0   55.0  275.0
All      467.5  220.0  687.5
Region     Date  North  South    All
0       2023-01  247.5  165.0  412.5
1       2023-02  220.0   55.0  275.0
2           All  467.5  220.0  687.5


#### Merging Joning and Concat

Goal	Use This	Key Characteristic 

Stacking data (top-to-bottom)	concat()	Simple "gluing" by index/columns.

Linking different info by ID	merge()	Flexible; can join on any column.

Linking data by the Index	join()	Convenient and fast for indexed data.

##### Merging

In [4]:

## It is just like noremal SQL JOIN operations
import pandas as pd
df_orders = pd.DataFrame({
    "OrderID": [101, 102, 103, 104],
    "CustomerID": [1, 2, 2, 5],  # Note: CustomerID 5 doesn't exist in df_customers
    "Amount": [250, 150, 300, 400]
})
df_customers = pd.DataFrame({
    "CustomerID": [1, 2, 3, 4],
    "Name": ["Alice", "Bob", "Charlie", "David"],
    "City": ["Mumbai", "Delhi", "Bangalore", "Chennai"]
})
pd.merge(df_customers,df_orders,on="CustomerID",how="inner")


,CustomerID,Name,City,OrderID,Amount
0,1,Alice,Mumbai,101,250
1,2,Bob,Delhi,102,150
2,2,Bob,Delhi,103,300


#### Concat

In [5]:

import pandas as pd

# Base Data
df_A = pd.DataFrame({'ID': [1, 2], 'Data': ['A1', 'A2']})
df_B = pd.DataFrame({'ID': [3, 4], 'Data': ['B1', 'B2']})
df_C = pd.DataFrame({'City': ['NY', 'LA']}, index=[0, 1])

# --- VERTICAL (axis=0) ---
# Adding more rows. Use ignore_index to get a fresh 0, 1, 2, 3 index.
#When you concatenate two DataFrames vertically, they each bring their original "row labels" (indices) with them. If both DataFrames start at 0, your final table will have duplicate index numbers ($0, 1, 0, 1$).
vertical = pd.concat([df_A, df_B], ignore_index=True)
display(vertical)
# --- HORIZONTAL (axis=1) ---
# Adding more columns side-by-side. Matches by the row index.
horizontal = pd.concat([df_A, df_C], axis=1)
display(horizontal)

,ID,Data
0,1,A1
1,2,A2
2,3,B1
3,4,B2


,ID,Data,City
0,1,A1,NY
1,2,A2,LA


#### Join

In [6]:
# Left Table: Employees
#another way of creating dataframe
emp = pd.DataFrame(columns=["Name"],
    data= ['Alice', 'Bob', 'Charlie'], 
    index=[101, 102, 103] # Employee ID as Index
)

# Right Table: Departments
dept = pd.DataFrame({
    'Dept': ['HR', 'IT', 'Sales']}, 
    index=[101, 102, 105] # Note 105 is a new ID
)

# JOIN matches based on the Index (ID)
# 'left' means keep all employees, even if they don't have a department
result = emp.join(dept, how='left')

print(result)
# Output:
#        Name  Dept
# 101   Alice    HR
# 102     Bob    IT
# 103 Charlie   NaN  <-- Charlie has no dept in the right table

        Name Dept
101    Alice   HR
102      Bob   IT
103  Charlie  NaN
